In [2]:
data_path = "./Results/"

In [72]:
threat_lexicon = data_path + "shapley_lexicon_1.txt"
derogation_lexicon = data_path + "shapley_lexicon_2.txt"
animosity_lexicon = data_path + "shapley_lexicon_3.txt"
predjudice_discussion_lexicon = data_path +  "shapley_lexicon_4.txt"

threat_lexicon_aug = data_path + "shapley_lexicon_1_aug.txt"
derogation_lexicon_aug = data_path + "shapley_lexicon_2_aug.txt"
animosity_lexicon_aug = data_path + "shapley_lexicon_3_aug.txt"
predjudice_discussion_lexicon_aug = data_path + "shapley_lexicon_4_aug.txt"

In [73]:
label_words_map = {
    "Threat":threat_words,
    "Derogation":derogation_words,
    "Animosity":animosity_words,
    "Prejudice discussion":predjudice_discussion_words
}

label_aug_file_map = {
    "Threat": threat_lexicon_aug,
    "Derogation": derogation_lexicon_aug,
    "Animosity": animosity_lexicon_aug,
    "Prejudice discussion": predjudice_discussion_lexicon_aug
}

In [17]:
threat_words = []
derogation_words = []
animosity_words = []
predjudice_discussion_words = []
def get_lexicon_words_list():
    # read threat words
    f = open(threat_lexicon)
    for word in f.read().split():
        threat_words.append(word)
        
    # read derogation words
    f = open(derogation_lexicon)
    for word in f.read().split():
        derogation_words.append(word)
        
    # read animosity words
    f = open(animosity_lexicon)
    for word in f.read().split():
        animosity_words.append(word)
        
    # read prejudice discussion words
    f = open(predjudice_discussion_lexicon)
    for word in f.read().split():
        predjudice_discussion_words.append(word)

get_lexicon_words_list()

In [18]:
print(len(threat_words))
print(len(derogation_words))
print(len(animosity_words))
print(len(predjudice_discussion_words))

1853
4778
4158
2183


In [27]:
print(label_words_map.get('sdaw'))

None


In [30]:
import random
def get_four_random_label_words(label):
    if(label_words_map.get(label) != None):
        res = random.choices(label_words_map[label], k=4)
        return res[0],res[1],res[2],res[3]
    else:
        print("label not found")
        return None;

In [57]:
import random
def get_random_label():
    return random.choices(["Threat", "Derogation", "Animosity", "Prejudice discussion"], k=1)[0]

In [58]:
print(get_random_label())

Threat


In [61]:
get_four_random_label_words(get_random_label())

('ruined', 'full', '>', 'warfare')

In [20]:
def write_list_to_file(lst, filename):
    # input text file
    inputFile = filename

    # Opening the given file in write mode
    with open(inputFile, 'w') as filedata:

        # Traverse in each element of the input list 
        for item in lst:

            # Writing each element of the list into the file
            # Here “%s\n” % syntax is used to move to the next line after adding an item to the file.
            filedata.write("%s\n" % item)

In [21]:
# write_list_to_file(["test", "Hi", "Lol"], "test_write.txt")

In [38]:
def get_label_prompt(label, word1, word2, word3, word4):
    # define a function that takes as input three samples and generates the prompt
    # that we should pass to the GPT-3 language model for completion.
    description = f"Each item in the following list contains a {label} word"
    prompt = (f"{description}\n"
            f"Item: {word1}.\n"
            f"Item: {word2}.\n"
            f"Item: {word3}.\n"
            f"Item: {word4}.\n"
            f"Item:")
    return prompt


In [40]:
a,b,c,d = get_four_random_label_words("Threat")
print(get_label_prompt("Threat", a,b,c,d))

Each item in the following list contains a Threat word
Item: talk.
Item: woman.
Item: burned.
Item: bump.
Item:


In [ ]:
from transformers import pipeline, AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer, TrainerCallback, AutoModelForCausalLM, AutoModelForSeq2SeqLM
max_input_len = 30
tokenizer = AutoTokenizer.from_pretrained("EleutherAI/gpt-j-6B")
model = AutoModelForCausalLM.from_pretrained("EleutherAI/gpt-j-6B")

In [62]:
# block of code to generate Synthetic Tweets!
# define the number of synthetic samples to generate
n = 100
new_texts = []
# new_labels = []

iter = 0
while iter < n:
    # get random label
    random_label = get_random_label()
    
    # get four random words
    a,b,c,d = get_four_random_label_words(random_label)
    
    # create the prompt
    prompt = get_label_prompt(random_label, a, b, c, d)

    # generate text using GPT-J model
    input_ids = tokenizer(prompt, return_tensors="pt").input_ids
    
#     print('before_gentokens')
    gen_tokens = model.generate(input_ids, do_sample=True, temperature=0.9, max_length=100,)
#     print('after_gentokens')
    gen_text = tokenizer.batch_decode(gen_tokens)[0]
    
    # the generated output will be in the form "<text> (Sentiment: <label>)"
    data = gen_text.split('\n')[3].strip('Item: ')
    
    if len(data) < 2:
        # the format of the response is invalid
        continue

    text = data[0]
    print('text', text);
    label_words_map[random_label].append(text)
    iter += 1


NameError: name 'tokenizer' is not defined

In [74]:

write_list_to_file(threat_words, label_aug_file_map["Threat"]) # write list to the new file!
write_list_to_file(derogation_words, label_aug_file_map["Derogation"]) # write list to the new file!
write_list_to_file(animosity_words, label_aug_file_map["Animosity"]) # write list to the new file!
write_list_to_file(predjudice_discussion_words, label_aug_file_map["Prejudice discussion"]) # write list to the new file!